In [1]:
%load_ext autoreload
%autoreload 2
import jieba
from jieba import analyse
import transformers as ts
import pandas as pd
import numpy as np
import torch
import json
from sklearn.model_selection import train_test_split
from ark_nlp.model.ner.global_pointer_bert import Dataset as arkData
from ark_nlp.model.ner.global_pointer_bert import Tokenizer,GlobalPointerBertConfig,GlobalPointerBert,get_default_model_optimizer,Task,Predictor
from torch.utils.data import DataLoader,Dataset

In [2]:
df_train=pd.read_json('./datasets/train.json')
lens=len(df_train)
df_train.head(3)

,text,ner_list,type,doc_id
0,血管内皮生长因子表达水平与宫颈癌患者预后的关系,"[{'mention': '血管内皮生长因子表达水平与宫颈癌患者预后的关系', 'label...",Title,2500
1,目的探讨地尔硫卓在治疗扩张型心肌病中的临床疗效。方法将2014年9月至2017年12月在红河...,"[{'mention': '探讨地尔硫卓在治疗扩张型心肌病中的临床疗效', 'label':...",Abstract,2501
2,地尔硫卓在治疗扩张型心肌病中的临床疗效观察,"[{'mention': '地尔硫卓在治疗扩张型心肌病中的临床疗效观察', 'label':...",Title,2501


In [3]:
#用于W2NER
result=[]
for i in range(0,lens):
    if df_train.loc[i,'type'] == 'Title':
        text=df_train.loc[i,'text'].replace(' ','')
        #token
        sentence=[char for char in text]
        #实体
        ner=[]
        for n in df_train.loc[i,'ner_list']:
            entity=n['mention'].replace(' ','')
            l=text.index(entity)
            r=l+len(entity)
            index=[j for j in range(l,r)]
            type=n['label'][0]
            ner.append({
                'index':index,
                'type':type,
            })
        #分词
        word=[]
        split_word=jieba.tokenize(text)
        for w in split_word:
            widx=[j for j in range(w[1],w[2])]
            word.append(widx)
        #组装
        result.append({
            'sentence':sentence,
            'ner':ner,
            'word':word,
        })
result_json=json.dumps(result,ensure_ascii = False)
with open('./datasets/w2ner/train.json','w',encoding='utf-8')as f:
    f.write(result_json)
    f.close()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\SUZIKA~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.052 seconds.
Prefix dict has been built successfully.


In [12]:
bigcat=[]
title_ner=[]
smallcat={
    'C':[],
    'I':[],
    'O':[],
    'P':[],
    'S':[],
}
for i in range(0,lens):
    if df_train.loc[i,'type'] == 'Title':
        newner=[]
        text=df_train.loc[i,'text'].replace(' ','')
        for ner in df_train.loc[i,'ner_list']:
            l=ner['label'][0]
            newbig={
                'text':ner['mention'],
                'label':l,
            }
            bigcat.append(newbig)
            
            smallcat[l].append({
                'text':ner['mention'],
                'label':ner['label'][2:],
            })
            
            subtext=ner['mention'].replace(' ','')
            sidx=text.find(subtext)
            if sidx == -1:
                continue
            eidx=sidx+len(subtext)
            newner.append({
                'start_idx':sidx,
                'end_idx':eidx,
                'type':ner['label'][0],
                'entity':subtext,
            })
            
        title_ner.append({
            'text':df_train.loc[i,'text'],
            'label':newner,
        })
    else:
        for ner in df_train.loc[i,'ner_list']:
            l=ner['label'][0]
            curtext=ner['mention']
            if len(curtext) > 200:
                todel=len(curtext)-198
                ll=todel//2
                r=todel-ll
                curtext=curtext[ll:-r]
            newbig={
                'text':curtext,
                'label':l,
            }
            bigcat.append(newbig)
            smallcat[l].append({
                'text':curtext,
                'label':ner['label'][2:],
            })
big_json=json.dumps(bigcat,ensure_ascii = False)
with open('./datasets/hierarchical/bigcat_train.json','w',encoding='utf-8') as f:
    f.write(big_json)
    f.close()
    
title_json=json.dumps(title_ner,ensure_ascii = False)
with open('./datasets/hierarchical/titlener_train.json','w',encoding='utf-8') as f:
    f.write(title_json)
    f.close()
    
for key in smallcat.keys():
    small_json=json.dumps(smallcat[key],ensure_ascii = False)
    with open('./datasets/hierarchical/'+key+'cat_train.json','w',encoding='utf-8') as f:
        f.write(small_json)
        f.close()

In [ ]:
ttt="采用抽签法将安阳市中医院心病一科2014年3月~2015年4月113例不稳定型心绞痛患者进行分组,对照组56例给予疏血通注射液6mL/d治疗,观察组57例给予疏血通注射液10mL/d治疗,观察两组临床疗效及不良反应"
analyse.extract_tags(ttt,topK=30)

In [5]:
tt=ts.BertTokenizer.from_pretrained('././../../model/macbert-base-chinese-medical-collation/')

In [34]:
ids=tt('乌拉地尔复合艾司洛尔控制[MASK]在高血压患者脊柱手术中的应用')['input_ids']
len(ids)

29

In [3]:
df_test=pd.read_json('./datasets/test.json')
lenst=len(df_test)
df_test.head(3)

,text,ner_list,type,doc_id
0,常染色体显性多囊肾病的临床问题及其肾脏替代治疗的选择,[],Title,500
1,,"[{'mention': '表明,在体外条件下,胃癌干细胞对5-氟尿嘧啶敏感性较低,推测其可...",Abstract,501
2,胃癌干细胞对5-氟尿嘧啶的敏感性,[],Title,501


In [39]:
sp=jieba.tokenize('RhoC的表达与肿瘤细胞分化、浸润深度未见相关性(P=0.977,P=0.141),与淋巴结转移、PTNM分期存在相关性(P=0.014,P=0.039)')
[word[0] for word in sp ]

['RhoC',
 '的',
 '表达',
 '与',
 '肿瘤',
 '细胞',
 '分化',
 '、',
 '浸润',
 '深度',
 '未见',
 '相关性',
 '(',
 'P',
 '=',
 '0.977',
 ',',
 'P',
 '=',
 '0.141',
 ')',
 ',',
 '与',
 '淋巴结',
 '转移',
 '、',
 'PTNM',
 '分期',
 '存在',
 '相关性',
 '(',
 'P',
 '=',
 '0.014',
 ',',
 'P',
 '=',
 '0.039',
 ')']

In [59]:
import re
def contains_alphanumeric(input_string):
    if re.search(r'[a-zA-Z0-9!@#$%^&*()~≥=();:,./]', input_string):
        return False
    else:
        return True

In [62]:
result=[]
for i in range(0,lens):
    if df_train.loc[i,'type'] == 'Title':
        text=df_train.loc[i,'text']
        if len(text) > 200:
                todel=len(text)-198
                ll=todel//2
                r=todel-ll
                text=text[ll:-r]
        text_words=jieba.tokenize(text)
        text_words=[words[0] for words in text_words]
        tlen=len(text_words)
        if tlen < 5:
            continue
        nmask=max(int(tlen*0.2),1)
        mask_idx=np.random.choice(np.arange(0,tlen),size=nmask,replace=False)
        for idx in mask_idx:
            if contains_alphanumeric(text_words[idx]):
                text_words[idx]=len(text_words[idx])*'[MASK]'
        result.append({
                'text':text,
                'masked':''.join(text_words)
            })
    if df_train.loc[i,'type'] == 'Abstract':
        for entity in df_train.loc[i,'ner_list']:
            text=entity['mention']
            if len(text) > 200:
                    todel=len(text)-198
                    ll=todel//2
                    r=todel-ll
                    text=text[ll:-r]
            text_words=jieba.tokenize(text)
            text_words=[words[0] for words in text_words]
            tlen=len(text_words)
            if tlen < 5:
                continue
            nmask=max(int(tlen*0.2),1)
            mask_idx=np.random.choice(np.arange(0,tlen),size=nmask,replace=False)
            for idx in mask_idx:
                if contains_alphanumeric(text_words[idx]):
                    text_words[idx]=len(text_words[idx])*'[MASK]'
            result.append({
                    'text':text,
                    'masked':''.join(text_words)
                })
for i in range(0,lenst):
    if df_test.loc[i,'type'] == 'Title':
        text=df_test.loc[i,'text']
        if len(text) > 200:
                todel=len(text)-198
                ll=todel//2
                r=todel-ll
                text=text[ll:-r]
        text_words=jieba.tokenize(text)
        text_words=[words[0] for words in text_words]
        tlen=len(text_words)
        if tlen < 5:
            continue
        nmask=max(int(tlen*0.2),1)
        mask_idx=np.random.choice(np.arange(0,tlen),size=nmask,replace=False)
        for idx in mask_idx:
            if contains_alphanumeric(text_words[idx]):
                text_words[idx]=len(text_words[idx])*'[MASK]'
        result.append({
                'text':text,
                'masked':''.join(text_words)
            })
    if df_test.loc[i,'type'] == 'Abstract':
        for entity in df_test.loc[i,'ner_list']:
            text=entity['mention']
            if len(text) > 200:
                    todel=len(text)-198
                    ll=todel//2
                    r=todel-ll
                    text=text[ll:-r]
            text_words=jieba.tokenize(text)
            text_words=[words[0] for words in text_words]
            tlen=len(text_words)
            if tlen < 5:
                continue
            nmask=max(int(tlen*0.2),1)
            mask_idx=np.random.choice(np.arange(0,tlen),size=nmask,replace=False)
            for idx in mask_idx:
                if contains_alphanumeric(text_words[idx]):
                    text_words[idx]=len(text_words[idx])*'[MASK]'
            result.append({
                    'text':text,
                    'masked':''.join(text_words)
                })
result_json=json.dumps(result,ensure_ascii = False)
with open('./datasets/fintune.json','w',encoding='utf-8')as f:
    f.write(result_json)
    f.close()

In [4]:
result=[]
for i in range(0,lenst):
    if df_train.loc[i,'type'] == 'Title':
        result.append(df_train.loc[i,'text'])
    if df_train.loc[i,'type'] == 'Abstract':
        for entity in df_train.loc[i,'ner_list']:
            result.append(entity['mention'])
for i in range(0,lenst):
    if df_test.loc[i,'type'] == 'Title':
        result.append(df_test.loc[i,'text'])
    if df_test.loc[i,'type'] == 'Abstract':
        for entity in df_test.loc[i,'ner_list']:
            result.append(entity['mention'])
with open('./datasets/sentence.txt','w',encoding='utf-8') as f:
    for r in result:
        f.write(r+'\n')
        f.close

In [7]:
import tokenizers
bwpt = tokenizers.BertWordPieceTokenizer()
filepath = "./datasets/sentence.txt" # 语料文件
#训练分词器
bwpt.train(
    files=[filepath],
    vocab_size=50000, # 这里预设定的词语大小不是很重要
    min_frequency=1,
    limit_alphabet=1000
)
# 保存训练后的模型词表
bwpt.save_model('./pretrained_models/')

['./pretrained_models/vocab.txt']

In [11]:
vocab1=[]
vocab2=[]
with open('../../model/macbert-base-chinese-medical-collation/vocab.txt','r',encoding='utf-8') as f1:
    for line in f1:
        vocab1.append(line)
    f1.close()
with open('./pretrained_models/vocab.txt','r',encoding='utf-8') as f2:
    for line in f2:
        vocab2.append(line)
    f2.close()

for v in vocab1:
    if not v in vocab2:
        vocab2.append(v)
len(vocab2)    

21128

In [16]:
len(vocab1)

21128

In [13]:
with open('./pretrained_models/vocab.txt','w',encoding='utf-8') as f2:
    for v in vocab2:
        f2.write(v)
    f2.close()

In [15]:
tt=ts.BertTokenizer(vocab_file='./pretrained_models/vocab.txt')
len(tt)

21128

In [4]:
testtt="小细胞肺癌术后化疗或放化疗后预防性脑照射的临床观察"
sp1=jieba.tokenize(testtt)
[w[0] for w in sp1]

['小细胞', '肺癌', '术后', '化疗', '或', '放化疗', '后', '预防性', '脑', '照射', '的', '临床', '观察']

In [5]:
jieba.load_userdict('./datasets/med_word.txt')
sp2=jieba.tokenize(testtt)
[w[0] for w in sp2]


['小细胞', '肺癌', '术后', '化疗', '或', '放化疗', '后', '预防性', '脑', '照射', '的', '临床', '观察']

In [7]:
[w for w in jieba.cut(testtt)]

['小细胞', '肺癌', '术后', '化疗', '或', '放化疗', '后', '预防性', '脑', '照射', '的', '临床', '观察']